In [1]:
import pandas as pd
import numpy as np


In [2]:
##Import the excel file 
sensor_df = pd.read_csv('104.csv')

#Down sample to every 100
sensor_df_smol = sensor_df[0:10000:100]
sensor_df_smol.head()

,sensor ID,norm_secs (secs),time (days)
0,63802,0.000000,39416.607535
100,415483,4.500004,39416.607587
200,161919,9.000008,39416.607639
300,943442,12.500011,39416.607679
400,194738,16.500015,39416.607726


In [3]:
#filter to adjadent gateways with a time difference larger than 3 seconds
diff_sensor_df = sensor_df_smol.diff()
#print(diff_sensor_df[40:60])
#print(sensor_df_smol[40:60])
backbone_sensor_df = sensor_df_smol[diff_sensor_df['norm_secs (secs)'] > 3]
backbone_sensor_df.head()

,sensor ID,norm_secs (secs),time (days)
100,415483,4.500004,39416.607587
200,161919,9.000008,39416.607639
300,943442,12.500011,39416.607679
400,194738,16.500015,39416.607726
500,202164,20.000018,39416.607766


In [35]:
print(backbone_sensor_df.shape)
print(sensor_df_smol.shape)
print(backbone_sensor_df['sensor ID'].nunique())
print(backbone_sensor_df['sensor ID'].drop_duplicates())
gateway_ids = backbone_sensor_df['sensor ID'].drop_duplicates()
print(gateway_ids.iloc[10])
#print(backbone_sensor_df['sensor ID'][0:10])
print(backbone_sensor_df[9:11])
print(backbone_sensor_df.iloc[10,1])
test = np.linspace(100,1000,10,dtype=int)
#index = np.linspace(100,(num_gates-1)*100,num_gates-1,dtype=int)
#print(index)
print(test)
print((test//100))
print(backbone_sensor_df.iloc[10][0])

(23, 3)
(100, 3)
15
100     415483
200     161919
300     943442
400     194738
500     202164
600     292567
700     333474
800     434898
1000    117554
1200    544844
1600    216898
1700    406748
2000    882682
2100    304380
2200    469168
Name: sensor ID, dtype: int64
216898
      sensor ID  norm_secs (secs)   time (days)
1000     117554         38.000034  39416.607975
1100     194738         41.500037  39416.608015
41.500036752881336
[ 100  200  300  400  500  600  700  800  900 1000]
[ 1  2  3  4  5  6  7  8  9 10]
194738.0


In [72]:
#function to check how many times gateways are next to each other and return average time 
def gateway_adjacent(gateway_x, gateway_y):
    #print(gateway_x)
    #print(gateway_y)
    
    count = 0
    time = 0
    size = backbone_sensor_df.shape[0]
    #print(size)
    for x in range(size):
        if (backbone_sensor_df.iloc[x][0] == gateway_x):
            #if it's the first index 
            if (x == 0):
                if (backbone_sensor_df.iloc[x+1][0] == gateway_y):
                    count += 1
                    time += abs(backbone_sensor_df.iloc[x+1][1] - backbone_sensor_df.iloc[x][1])
            
            #if it's a last index
            elif (x == size-1):
                if (backbone_sensor_df.iloc[x-1][0] == gateway_y):
                    count += 1
                    time += abs(backbone_sensor_df.iloc[x-1][1] - backbone_sensor_df.iloc[x][1])
            
            else:
                if (backbone_sensor_df.iloc[x+1][0] == gateway_y):
                    count += 1
                    time += abs(backbone_sensor_df.iloc[x+1][1] - backbone_sensor_df.iloc[x][1])
                elif (backbone_sensor_df.iloc[x-1][0] == gateway_y):
                    count += 1
                    time += abs(backbone_sensor_df.iloc[x-1][1] - backbone_sensor_df.iloc[x][1])
                    
    if (count >= 1): 
        return time/count
    else:
        return 0
    

In [73]:
#set up a matrix of time between adjacent gateways, this is a proxy for distance
num_gates = backbone_sensor_df['sensor ID'].nunique()
time_pairs = np.zeros((num_gates,num_gates))
gateway_ids = backbone_sensor_df['sensor ID'].drop_duplicates()

for x in range(num_gates):
    #times = []
    for y in range(num_gates):
        #find how many times x gateway is adjacent to y gateway 
        gateway_id_x = gateway_ids.iloc[y]
        gateway_id_y = gateway_ids.iloc[x]
        average = gateway_adjacent(gateway_id_x,gateway_id_y)

        #add to time_pairs[x][y]
        time_pairs[x][y] = average
        
        
        #times.append(abs(backbone_sensor_df.iloc[x][1] == backbone_sensor_df.iloc[y][1]))
        #time_pairs[x][y] = abs(backbone_sensor_df.iloc[x][1] - backbone_sensor_df.iloc[y][1])
    #set the values
    #timenp.mean(times)
        
time_pairs

array([[0.        , 4.50000399, 4.00000354, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        4.00000354, 0.        , 0.        , 0.        , 0.        ],
       [4.50000399, 0.        , 3.5000031 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [4.00000354, 3.5000031 , 0.        , 4.00000354, 0.        ,
        3.5000031 , 0.        , 3.5000031 , 4.00000354, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 4.00000354, 0.        , 3.5000031 ,
        0.        , 0.        , 0.        , 3.5000031 , 3.5000031 ,
        0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 3.5000031 , 0.        ,
        3.5000031 , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 

In [ ]:
#get the average velocity from the ground truth data 

In [ ]:
#calculate the distance matrix from the average time and average velocity 

In [ ]:
#pick three sensors to triangulate (the furthest apart ones)

In [ ]:
#derive all distances from the three sensors

In [ ]:
#plot the gateways 

In [ ]:
#compare gateway locations to ground truth 

In [ ]:
#backbone_sensor_df = sensor_df.loc(sensor_df['norm_secs (secs)'].diff() > 1)
#print(backbone_sensor_df)
#print(sensor_df.filter(sensor_df['norm_secs (secs)'].diff() > 1))
#backbone_sensor_df = sensor_df_smol.loc(diff_sensor_df['norm_secs (secs)'] > 1.0)
#backbone_sensor_df.head()

#print(sensor_df_smol[diff_sensor_df['norm_secs (secs)'] > 3])

##print(sensor_df['norm_secs (secs)'][100:110])
#print(sensor_df["sensor ID"][0])

#Get a pairwise matrix for the backbone gateways
num_gates = backbone_sensor_df.shape[0]
index = np.linspace(100,(num_gates-1)*100,num_gates-1,dtype=int)

#set up a matrix of time between gateways, this is a proxy for distance
time_pairs = np.ones((num_gates,num_gates))

for x in index:
    times = []
    for y in index:
        #get all the pairs
        if (backbone_sensor_df["sensor ID"][x+100] == backbone_sensor_df["sensor ID"][y]):
            times.append(abs(backbone_sensor_df["norm_secs (secs)"][x+100] - backbone_sensor_df["norm_secs (secs)"][y]))
        
        #set the distance values
        pairs[(x//100)-1][(y//100)-1] = np.mean(times)
        
#pairs

#if (backbone_sensor_df.iloc[x][1] == backbone_sensor_df.iloc[y][1]):